<a href="https://colab.research.google.com/github/krish04shah/Material-And-Energy-Balances/blob/main/IsopropanolRecycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!python -m pip install importnb --quiet
from importnb import Notebook
with Notebook():
    from che4 import Range, Comp, CompArray, RangeArray, DotDict, d2nt, Props


In [ ]:
import numpy as np
from scipy.optimize import root
import jax
import jax.numpy as jnp

In [ ]:
#rx1 C3H6 + H2O -> C3H7OH
#rx2 2C3H7OH -> (C3H7)O(C3H7) + H2O
#1. C3H6, 2. (C3H7)O(C3H7), 3. C3H7OH, 4. H2O

e = DotDict()

e.stoich_rx1 = np.array([-1., 0., 1., -1.])
e.stoich_rx2 = np.array([0., 1., -2., 1.])

e.alkeneF = 100. #mol/s
e.alkene_comp = np.array([1.,0. , 0. ,0.])

e.waterF = 95. #mol/s
e.water_comp = np.array([0., 0., 0., 1.])

e.etherF = Range(5., 0., e.alkeneF/2)
e.ether_comp = np.array([0.001, 0.9, 0.09, 0.009])

e.alcoholF = Range(50., 0., e.alkeneF)
e.alcohol_comp = np.array([0.001, 0.01, 0.94, 0.049])

e.ext_rxn1 = Range(e.alkeneF/2, 0., e.alkeneF)
e.ext_rxn2 = Range(e.alkeneF/4, 0., e.alkeneF/2)

e.WARFR = 12. # Water-Alkene Reactor Feed Rate

e.D1F = Range(e.WARFR*e.alkeneF, 0., 2.* e.WARFR*e.alkeneF)
e.D1F_comp = Comp([ 0.01 ,0.03, 0.06,0.9])

e.D1d = Range(e.WARFR*e.alkeneF/2, 0., 2.* e.WARFR*e.alkeneF)
e.D1d_comp = Comp([0.01,0.33, 0.33, 0.33])

e.D1B = Range(e.WARFR*e.alkeneF/2, 0., 2.* e.WARFR*e.alkeneF)
e.D1B_comp = Comp([0.0001,0.1,0.1,0.8])

e.decanterW = Range(e.WARFR*e.alkeneF, 0., 2.* e.WARFR*e.alkeneF)
e.decanterW_comp = np.array([ 0.00001 , 0.002, 0.018, 0.98])

e.D1K = np.array([100., 10., 5., 0.2])

In [ ]:
x0, x2nt, wrap, x2unk, const = d2nt(e)

In [ ]:
def eqs(d):

  feed = d.alkeneF*d.alkene_comp + d.waterF*d.water_comp
  rxns = d.stoich_rx1*d.ext_rxn1 + d.stoich_rx2*d.ext_rxn2
  prod = d.etherF*d.ether_comp + d.alcoholF*d.alcohol_comp

  return feed + rxns - prod # should be 0

In [ ]:
def eqs2(d):
    reactor_feed = d.alkeneF*d.alkene_comp + d.waterF*d.water_comp + d.D1B*d.D1B_comp + d.decanterW*d.decanterW_comp
    rxns = d.stoich_rx1*d.ext_rxn1 + d.stoich_rx2*d.ext_rxn2
    reactor_prod = d.D1F*d.D1F_comp

    WARFR_CONSTRAINT = (d.waterF + d.D1B + d.decanterW)/d.alkeneF - d.WARFR

    REACTOR_BALANCE = reactor_feed + rxns - reactor_prod

    D1_BALANCE = d.D1F*d.D1F_comp - d.D1B*d.D1B_comp - d.D1d*d.D1d_comp
    D1_VLE = d.D1d_comp/d.D1B_comp - d.D1K

    AZ_BALANCE = d.D1d*d.D1d_comp -  d.decanterW*d.decanterW_comp -  d.etherF*d.ether_comp - d.alcoholF*d.alcohol_comp

    return jnp.concatenate([REACTOR_BALANCE, D1_BALANCE, D1_VLE, AZ_BALANCE, jnp.atleast_1d(WARFR_CONSTRAINT)])

In [ ]:
wrapped_eqs2 = wrap(eqs2)

In [ ]:
sol = root(wrapped_eqs2, x0, jac = jax.jacobian(wrapped_eqs2))
print(sol)

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.328e-09  2.242e-11 ...  6.465e-12 -1.776e-15]
       x: [-1.481e+00  1.802e+00 ... -1.730e+00 -5.106e+00]
    nfev: 39
    njev: 3
    fjac: [[ 1.002e-07 -4.482e-11 ...  9.950e-03  2.228e-13]
           [-6.596e-08  3.293e-11 ...  5.127e-02 -1.314e-13]
           ...
           [-5.077e-02 -4.135e-02 ...  1.056e-01 -5.908e-10]
           [-6.552e-04 -1.908e-03 ... -9.267e-03  1.155e-10]]
       r: [-6.826e+00 -1.264e+00 ... -6.375e-01  1.216e-01]
     qtf: [ 3.814e-11 -2.420e-10 ... -5.214e-13  1.205e-13]


In [ ]:
x2unk(sol.x)._asdict()

{'etherF': Array(9.26195012, dtype=float64),
 'alcoholF': Array(85.83314497, dtype=float64),
 'ext_rxn1': Array(99.9049049, dtype=float64),
 'ext_rxn2': Array(9.19408656, dtype=float64),
 'D1F': Array(1200.09523964, dtype=float64),
 'D1F_comp': Array([8.72602370e-05, 1.53357353e-02, 2.03810648e-01, 7.80766356e-01],      dtype=float64),
 'D1d': Array(109.54926493, dtype=float64),
 'D1d_comp': Array([0.00086938, 0.08419038, 0.74648519, 0.16845505], dtype=float64),
 'D1B': Array(1090.5459747, dtype=float64),
 'D1B_comp': Array([8.69377219e-06, 8.41903833e-03, 1.49297039e-01, 8.42275229e-01],      dtype=float64),
 'decanterW': Array(14.4540253, dtype=float64)}